<a href="https://colab.research.google.com/github/NeginS98/Precipitation-Induced-Infiltration-Using-Machine-Learning-and-Uncertainty-Framework/blob/main/final_Upstream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import random
import os
import glob

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import metrics
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten,BatchNormalization
from keras.callbacks import EarlyStopping
from keras import optimizers

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm *

# **Read data**

In [ ]:
ghale_shahrokh = pd.read_csv('/content/drive/MyDrive/Upstream_models/ghale_sha_water_balance_result.csv').iloc[:, 0:7]

In [ ]:
ghale_shahrokh.rename(columns={'month_num':'month', 'temp_mean': 'tempreture'}, inplace=True)

# **Model LSTM ghale_shahrokh**

In [ ]:
x = ghale_shahrokh.drop(['year', 'month', 'infilteration'], axis=1)
y = ghale_shahrokh.infilteration

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units = 32, return_sequences = True, input_shape = (x.shape[1], 1)))
regressor.add(Dropout(0.1))
regressor.add(LSTM(units = 32, return_sequences = True))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(x_train, y_train, epochs = 500, batch_size = 32,
              validation_data=(x_test,y_test),
              callbacks=EarlyStopping(monitor='val_loss',patience=5))
y_pred = regressor.predict(x_test)
eval_dict = {'test size': 0.25,
              'MAE': metrics.mean_absolute_error(y_test, y_pred),
              'MSE':metrics.mean_squared_error(y_test, y_pred),
              'RMSE':np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
              'R2 Score':metrics.r2_score(y_test, y_pred),
}

In [ ]:
eval_dict = {'test size': 0.25,
              'MAE':'%.3f' % metrics.mean_absolute_error(y_test, y_pred),
              'MSE':'%.3f' %metrics.mean_squared_error(y_test, y_pred),
              'RMSE':'%.3f' %np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
              'R2 Score':'%.3f' %metrics.r2_score(y_test, y_pred),
              'R':'%.3f' % np.sqrt(metrics.r2_score(y_test, y_pred))
              }
eval_dict

In [ ]:
val = []
key = []
for i in eval_dict.values():
  val.append(i)
for j in eval_dict.keys():
  key.append(j)
eval_df = pd.DataFrame()
eval_df.insert(0, 'None', key)
eval_df.insert(1, 'value', val)
header_row = eval_df.iloc[0]
eval_df = pd.DataFrame(eval_df.values[1:], columns=header_row)

# **Prediction By LSTM**

**Data**

In [ ]:
path_126='/content/drive/MyDrive/Upstream_basin_models/Upstream_126.xlsx'
path_245='/content/drive/MyDrive/Upstream_basin_models/Upstream_245.xlsx'
path_585='/content/drive/MyDrive/Upstream_basin_models/Upstream_585.xlsx'

df_ssp126 = pd.read_excel(path_126, 'ghale_shahrokh').dropna(axis=0)
df_ssp245 = pd.read_excel(path_245, 'ghale_shahrokh').dropna(axis=0)
df_ssp585 = pd.read_excel(path_585, 'ghale_shahrokh').dropna(axis=0)

In [ ]:
date = df_ssp126.drop(['sub_basin', 'month_name',  'temp_126', 'pr_126',
       'discharge', 'AET'], axis=1)

In [ ]:
x_pred_126 = df_ssp126.drop({'sub_basin', 'year', 'month_name', 'month'}, axis=1)
x_pred_126.rename(columns={'temp_126':'tempreture',	'pr_126':'pr' ,	'discharge':'Runoff'}, inplace=True)
x_pred_245 = df_ssp245.drop({'sub_basin', 'year', 'month_name', 'month'}, axis=1)
x_pred_245.rename(columns={'temp_245':'tempreture',	'pr_245':'pr' ,	'discharge':'Runoff'}, inplace=True)
x_pred_585 = df_ssp585.drop({'sub_basin', 'year', 'month_name', 'month'}, axis=1)
x_pred_585.rename(columns={'temp_585':'tempreture',	'pr_585':'pr' ,	'discharge':'Runoff'}, inplace=True)

In [ ]:
regressor.fit(x , y)
y_pred_126 = model.predict(x_pred_126)
y_pred_245 = model.predict(x_pred_245)
y_pred_585 = model.predict(x_pred_585)

In [ ]:
rcParams['figure.figsize'] = 10, 3
plt.plot(y_pred_126)
plt.plot(y_pred_245)
plt.plot(y_pred_585)
plt.legend(['SSP126', 'SSP245', 'SSP585'])
plt.ylabel('Infilterarion')


In [ ]:
len(y_pred_126), len(y_pred_245), len(y_pred_585)

In [ ]:
result_final = pd.DataFrame({
    'year': date.year,
    'month': date.month,
    'scenario_126': y_pred_126,
    'scenario_245': y_pred_245[0:300],
    'scenario_585': y_pred_585[0:300],
})
result_final['scenario_126'] = result_final['scenario_126'].apply(lambda x : x if x > 0 else 0)
result_final['scenario_245'] = result_final['scenario_245'].apply(lambda x : x if x > 0 else 0)
result_final['scenario_585'] = result_final['scenario_585'].apply(lambda x : x if x > 0 else 0)